In [3]:
pip install opencv-python opencv-python-headless numpy

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 660.6 kB/s eta 0:00:59
   ---------------------------------------- 0.0/38.8 MB 393.8 kB/s eta 0:01:39
   ---------------------------------------- 0.0/38.8 MB 393.8 kB/s eta 0:01:39
   ---------------------------------------- 0.1/38.8 MB 272.3 kB/s eta 0:02:23
   ---------------------------------------- 0.1/38.8 MB 350.1 kB/s eta 0:01:51
   ---------------------------------------- 0.1/38.8 MB 448.2 kB/s eta 0:01:27
   ---------------------------------------- 0.2/38.8 MB 437.1 kB/s eta 0:01:29
   ---------------------------------------- 0.2/38.8 MB 445.2 kB/s eta 0:01:27
   ---------------------------------------- 0.2/38.8 MB 491.0 kB/s eta 0:01:19
   ---------------------------------------- 0.3/38.8 MB 524.0 kB/s eta 0:01:14
   ---------------------------------------- 0.3/38.8 MB 636.4 kB/s eta

In [5]:
import cv2
import numpy as np

In [43]:
vid = cv2.VideoCapture(0) 
  
if not vid.isOpened():
    raise IOError("Cannot open webcam")

while(True): 

    # Capture the video frame 
    # by frame 
    ret, frame = vid.read() 
    
    if not ret:  # Check if a frame was successfully read
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Display the resulting frame 
    cv2.imshow('frame', frame)
    #cv2_imshow(frame)
      
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows()

In [45]:
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows()

In [ ]:
# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Define boundary boxes for desks
desk_boundaries = {
    "desk1": (50, 50, 200, 200),  # (x1, y1, x2, y2)
    "desk2": (300, 50, 450, 200),
    # Add more desks as needed
}

def draw_boundary_boxes(frame):
    for desk, (x1, y1, x2, y2) in desk_boundaries.items():
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, desk, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

def is_outside_boundary(box, boundary):
    x, y, w, h = box
    bx1, by1, bx2, by2 = boundary
    if x < bx1 or x + w > bx2 or y < by1 or y + h > by2:
        return True
    return False

def process_frame(frame):
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "person":
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            for desk, boundary in desk_boundaries.items():
                if is_outside_boundary((x, y, w, h), boundary):
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    cv2.putText(frame, "Alert!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                else:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

cap = cv2.VideoCapture(0)  # Change 0 to your camera source

while True:
    ret, frame = cap.read()
    if not ret:
        break
    draw_boundary_boxes(frame)
    process_frame(frame)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
